In [4]:
import scipy.linalg
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pre-Processing

## Data Importing

In [5]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [6]:
path = '/content/drive/MyDrive/O4DS - Project Work/goodreads_cleaned.csv'
#path = 'DATA/goodreads_cleaned.csv'

In [7]:
df = pd.read_csv(path, sep = ";")
df

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,18245960,5
1,8842281e1d1347389f2ab93d60773d4d,16981,3
2,8842281e1d1347389f2ab93d60773d4d,28684704,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,4
...,...,...,...
899995,b9450d1c1f97f891c392b1105959b56e,11832081,3
899996,b9450d1c1f97f891c392b1105959b56e,16095092,3
899997,b9450d1c1f97f891c392b1105959b56e,8430896,4
899998,b9450d1c1f97f891c392b1105959b56e,12275680,4


## Data Exploration

In [8]:
df.user_id.value_counts().describe()

count    12188.000000
mean        73.843124
std        103.860677
min          1.000000
25%         14.000000
50%         37.000000
75%         92.000000
max       1815.000000
Name: user_id, dtype: float64

In [9]:
df.book_id.value_counts().describe()

count    25474.000000
mean        35.330141
std         67.222413
min          1.000000
25%         10.000000
50%         17.000000
75%         34.000000
max       1734.000000
Name: book_id, dtype: float64

## Data Cleaning

In [10]:
df['book_id_count'] = df.groupby('book_id')['book_id'].transform('count')
df['user_id_count'] = df.groupby('user_id')['user_id'].transform('count')
df.book_id_count.quantile(0.9)

457.0

In [11]:
book_quantile = 0.9
user_quantile = 0.25

df = df.loc[(df.book_id_count >= df.book_id.value_counts().quantile(book_quantile)) & (df.user_id_count >= df.user_id.value_counts().quantile(user_quantile)),:]

In [12]:
df.shape

(425794, 5)

## Data Pivoting

In [13]:
df = pd.pivot_table(df, columns="book_id", index="user_id", values="rating")
df.head(10)

book_id,1,2,3,5,6,11,34,295,320,343,350,662,667,830,865,890,902,930,960,968,1103,1232,1420,1617,1618,1622,1845,1852,1885,1934,1953,2052,2156,2165,2187,2493,2526,2612,2623,2657,2744,2839,2956,2998,3008,3431,3473,3636,3682,3758,...,30269126,30312891,30325011,30415154,30555488,30633337,30653853,30687916,30688435,30724132,30731416,30747137,30809689,30821598,30831912,30839185,30969741,31140847,31145133,31145148,31176886,31423196,31450752,31450852,31450908,31451174,31538614,31538635,31538647,31931941,31952703,32075662,32075671,32078787,32571395,32796253,32848471,33140405,33151805,33232571,33280872,33288638,33385229,33643994,34044126,34076952,34273458,35247769,35404657,35504431
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000a1016fda6008d1edbba720ca00851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011e1a9112b3d798702ef5b20bbf35b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019de4561419b7543238e0979f2f33e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00204424763e8233c5f53f0729f2304f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00214d8b0a020837cccf5f41eb563037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00238d8a4c276c47f5d5e242f54a8f28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002a023d3de233b4bd3ec4fc3e9c581a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002e063d40ae0107a59d8f9c1aa7a423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Convert to an array to work with the matrix

In [14]:
df.reset_index(drop=True)
df

book_id,1,2,3,5,6,11,34,295,320,343,350,662,667,830,865,890,902,930,960,968,1103,1232,1420,1617,1618,1622,1845,1852,1885,1934,1953,2052,2156,2165,2187,2493,2526,2612,2623,2657,2744,2839,2956,2998,3008,3431,3473,3636,3682,3758,...,30269126,30312891,30325011,30415154,30555488,30633337,30653853,30687916,30688435,30724132,30731416,30747137,30809689,30821598,30831912,30839185,30969741,31140847,31145133,31145148,31176886,31423196,31450752,31450852,31450908,31451174,31538614,31538635,31538647,31931941,31952703,32075662,32075671,32078787,32571395,32796253,32848471,33140405,33151805,33232571,33280872,33288638,33385229,33643994,34044126,34076952,34273458,35247769,35404657,35504431
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000a1016fda6008d1edbba720ca00851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011e1a9112b3d798702ef5b20bbf35b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019de4561419b7543238e0979f2f33e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00204424763e8233c5f53f0729f2304f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00214d8b0a020837cccf5f41eb563037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff3a250fbc018ad2c2c2d45c86734da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff7bfd82b89fa347edfe9a82ac0c61b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [15]:
data_matrix = df.to_numpy(na_value=np.nan)
print(data_matrix)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan  5. nan ... nan nan nan]]


In [16]:
# Check how to get the index of not empty values
idx = np.argwhere(~np.isnan(data_matrix))
print(idx)

[[   0  419]
 [   0  427]
 [   0  495]
 ...
 [9152 2452]
 [9152 2475]
 [9152 2559]]


In [17]:
np.unique(data_matrix[idx[:,0], idx[:,1]])

array([0., 1., 2., 3., 4., 5.])

# Frank-Wolfe - standard algorithm

- Should we feed $\delta$ to the FW algorithm or should it be defined based on the dimensions of the data?
- Which is the correct objective function?
- Initialize with random matrix of integers from 1 to 5 or with zeros matrix?

In [18]:
from scipy import sparse
from scipy import stats

In [19]:
def FW_objective_function(diff_vec):
    return 0.5*(np.power(diff_vec,2).sum())
    #return 0.5 * np.linalg.norm(diff_vec, 2)**2

def FrankWolfe(X, objective_function, delta, printing_res = True, Z_init = None, max_iter = 150, patience = 1e-3):
    '''
    :param X: sparse matrix with ratings and 'empty values', rows - users, columns - books.
    :param objective_function: objective function that we would like to minimize with FW
    :param Z_init: In case we want to initialize Z with a known matrix, if not given Z_init will be a zeros matrix
    :param max_iter: max number of iterations for the method
    :param patience: once reached this tolerance provide the result
    :return: Z: matrix of predicted ratings - it should be like X but with no 'empty values'
            accuracy: difference between original values (X) and predicted ones (Z)
    '''

    # Get X indexes for not empty values
    idx_ratings = np.argwhere(X != 0)
    #idx_ratings = np.argwhere(~np.isnan(X))
    idx_rows = idx_ratings[:,0]
    idx_cols = idx_ratings[:,1]

    # Initialize Z -- think about a good init
    if Z_init is not None:
        Z = Z_init
    else:
        #Z = np.random.randint(1, 6, size=X.shape)
        #Z = Z.astype(float)
        Z = np.zeros(X.shape)

    # Create vectors with the not empty features of the sparse matrix
    X_rated = X[idx_rows, idx_cols]
    Z_rated = Z[idx_rows, idx_cols]
    diff_vec = Z_rated - X_rated

    # choose an appropriate delta
    delta = delta

    diff_err = patience + 1
    err = objective_function(diff_vec)
    it = 0
    while (diff_err > patience) and (it < max_iter):

        # Gradient
        grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))

        # SVD
        u_max, s_max, v_max = sparse.linalg.svds(grad, k = 1, which='LM')   # Compute k = 1 singular values, starting from the largest (which = 'LM')

        # Update
        update_Z = -delta*np.outer(u_max,v_max)     # Zk_tilde in the theory

        #alpha - as studied in class
        alpha_k = 2/(it+2)
        Z = (1-alpha_k)*Z + alpha_k*update_Z

        # Error
        diff_vec = Z[idx_rows, idx_cols] - X_rated
        new_err = objective_function(diff_vec)

        # Improvement at this iteration
        diff_err = np.abs(err - new_err)
        err = new_err

        if printing_res == True:
          print('Iteration:', it, 'Err:', err, 'Diff err:', diff_err)

        # Count iteration
        it += 1
    return Z, update_Z, err

We build a smaller matrix for testing the FW alg, then we will apply it to our data

In [36]:
# Create a random sparse matrix for testing
rvs = stats.randint(1,6).rvs
X_test = sparse.random(1500, 2000,              # shape of the sparse matrix
            density = 0.05,             # density of the sparse matrix
            dtype = np.int32,           # data type
            data_rvs=rvs).toarray()     # distribution

#Normalize the values
X_test_norm = X_test/5

In [39]:
pred_ratings, loss = FrankWolfe(X_test_norm, FW_objective_function, delta = 7250, max_iter=1000, patience=1e-7)

(150000,)
[0.2 0.8 0.2 ... 0.6 0.2 1. ]
Iteration: 0 Err: 1025593.8168087436 Diff err: 992542.2368087437
Iteration: 1 Err: 338772.33271484333 Diff err: 686821.4840939003
Iteration: 2 Err: 94045.47792366572 Diff err: 244726.8547911776
Iteration: 3 Err: 248134.4140444225 Diff err: 154088.93612075679
Iteration: 4 Err: 1629809.6057842993 Diff err: 1381675.1917398768
Iteration: 5 Err: 397151.23028288776 Diff err: 1232658.3755014115
Iteration: 6 Err: 746230.989675663 Diff err: 349079.7593927752
Iteration: 7 Err: 276544.4780241303 Diff err: 469686.51165153267
Iteration: 8 Err: 439363.6927645071 Diff err: 162819.2147403768
Iteration: 9 Err: 206591.27460556748 Diff err: 232772.4181589396
Iteration: 10 Err: 293957.9836481694 Diff err: 87366.7090426019
Iteration: 11 Err: 162649.81547662866 Diff err: 131308.16817154072
Iteration: 12 Err: 214083.68577863427 Diff err: 51433.87030200561
Iteration: 13 Err: 133364.6870501753 Diff err: 80718.99872845897
Iteration: 14 Err: 165657.87339204756 Diff err: 32

KeyboardInterrupt: ignored

In [ ]:
pred_ratings*5

array([[0.01674913, 0.01434759, 0.01507895, ..., 0.01425873, 0.01171163,
        0.01912285],
       [0.02097993, 0.01797177, 0.01888787, ..., 0.01786046, 0.01466997,
        0.02395326],
       [0.01818798, 0.01558014, 0.01637433, ..., 0.01548364, 0.01271773,
        0.02076562],
       ...,
       [0.01870729, 0.01602499, 0.01684185, ..., 0.01592574, 0.01308085,
        0.02135853],
       [0.01441318, 0.01234658, 0.01297594, ..., 0.01227011, 0.01007824,
        0.01645585],
       [0.01650178, 0.01413571, 0.01485627, ..., 0.01404816, 0.01153867,
        0.01884045]])

In [ ]:
idx_ratings = np.argwhere(~np.isnan(X_test))
idx_rows = idx_ratings[:,0]
idx_cols = idx_ratings[:,1]
pred_ratings[idx_rows,idx_cols]*5

array([0.01674913, 0.01434759, 0.01507895, ..., 0.01404816, 0.01153867,
       0.01884045])

#### Our data prediction

In [20]:
new_data = np.nan_to_num(data_matrix, 0)

In [42]:
pred_ratings, loss = FrankWolfe(new_data, FW_objective_function, delta = 43200, max_iter=10000, patience=1e-5)

(412898,)
[5. 4. 5. ... 5. 2. 3.]
Iteration: 0 Err: 169521072.7652239 Diff err: 166171807.7652239
Iteration: 1 Err: 128322206.78089207 Diff err: 41198865.98433182
Iteration: 2 Err: 59513961.45147687 Diff err: 68808245.3294152
Iteration: 3 Err: 49811164.41752395 Diff err: 9702797.033952922
Iteration: 4 Err: 30249657.409918882 Diff err: 19561507.00760507
Iteration: 5 Err: 26538908.856101528 Diff err: 3710748.553817354
Iteration: 6 Err: 18209640.242947124 Diff err: 8329268.613154404
Iteration: 7 Err: 16855018.78428113 Diff err: 1354621.458665993
Iteration: 8 Err: 12632848.066807121 Diff err: 4222170.71747401
Iteration: 9 Err: 12131203.513994992 Diff err: 501644.55281212926
Iteration: 10 Err: 9659089.161567688 Diff err: 2472114.352427304
Iteration: 11 Err: 9500811.138292564 Diff err: 158278.0232751239
Iteration: 12 Err: 7894403.550649143 Diff err: 1606407.5876434213
Iteration: 13 Err: 7889573.968425838 Diff err: 4829.582223304547


KeyboardInterrupt: ignored

# Frank-Wolfe In-face

In [14]:
a = np.array([1,2,1,2,1])
b = np.array([1,2,3,4,5])
np.multiply(a,b)

array([1, 4, 3, 8, 5])

In [21]:
from numpy import linalg as LA

In [22]:
def FW_objective_function(diff_vec):
    return 0.5*(np.power(diff_vec,2).sum())
    #return 0.5 * np.linalg.norm(diff_vec, 2)**2
    
def alpha_binary_search(Zk, Dk, delta, max_value = 1, min_value = 0, tol = 0.05):
        
    #Inizialization
    
    best_alpha = (max_value - min_value) / 2
    
    testing_matrix = Zk + best_alpha * Dk
    
    testing_mat_nuclear_norm = LA.norm(testing_matrix, ord = 'nuc')
    
    #Binary Search
    
    while testing_mat_nuclear_norm <= delta and (max_value - min_value) >= tol:
        
        min_value = best_alpha
        
        best_alpha = (max_value - min_value) / 2

        testing_matrix = Zk + best_alpha * Dk
    
        testing_mat_nuclear_norm = LA.norm(testing_matrix, ord = 'nuc')     
        
    return best_alpha

def FW_inface(X, objective_function, delta, Z_init = None, max_iter=150, patience=1e-3):
    '''
    :param X: sparse matrix with ratings and 'empty values', rows - users, columns - books.
    :param objective_function: objective function that we would like to minimize with FW.
    :param Z_init: In case we want to initialize Z with a known matrix, if not given Z_init will be a zeros matrix.
    :param max_iter: max number of iterations for the method.
    :param patience: once reached this tolerance provide the result.
    :return: Z: matrix of predicted ratings - it should be like X but with no 'empty values'
            loss: difference between original values (X) and predicted ones (Z).
    '''

    # Get X indexes for not empty values
    idx_ratings = np.argwhere(X != 0)
    #idx_ratings = np.argwhere(~np.isnan(X))
    idx_rows = idx_ratings[:,0]
    idx_cols = idx_ratings[:,1]

    # choose an appropriate delta

    # Initialize Z_{-1}
    if Z_init is not None:
        Z = Z_init
    else:
        Z = np.zeros(X.shape)

    # Create vectors with the not empty features of the sparse matrix
    X_rated = X[idx_rows, idx_cols]
    Z_rated = Z[idx_rows, idx_cols]
    diff_vec = Z_rated - X_rated

    # Initial gradient and Z0
    grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))
    u_max, s_max, v_max = sparse.linalg.svds(grad, k = 1, which='LM')
    Z = -delta*np.outer(u_max,v_max)
    Z_rated = Z[idx_rows, idx_cols]

    # Initialize lower bound on the optimal objective function (f*)
    diff_vec = Z_rated - X_rated
    new_low_bound = np.max((objective_function(diff_vec) + np.multiply(diff_vec,Z_rated)),0)

    # Set L and D constants and gamma1, gamma2 constraints
    L = 1
    D = 2*delta
    gamma1 = 0
    gamma2 = 1

    # Compute first iteration thin SVD
    grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))
    r_grad = sparse.csgraph.structural_rank(grad)   # Compute rank of the gradient sparse matrix to find thin SVD size
    U_thin, D_thin, Vh_thin = sparse.linalg.svds(grad, k = 1, which='LM')   # Compute k = rank singular values # replaced r_grad with 1


    # Additional needed parameters
    diff_objective = patience + 1
    objective = objective_function(diff_vec)
    it = 0
    while (diff_objective > patience) and (it < max_iter):

        # Lower bound update
        low_bound = new_low_bound

        # In-face direction with the away step strategy: two calculations depending of where Z lies within the feasible set
        if D_thin.sum() == delta: # Z in border (sum of singular values == radious of feasible set)
            G = 0.5(Vh_thin.dot(grad.T.dot(U_thin)) + U_thin.T.dot(grad.dot(Vh_thin.T)))
            u = sparse.linalg.eigs(G, k = 1, which = 'SM')#unitary eigenvector corresponding to smallest eigenvalue of G
            M = np.outer(u,u)
            update_Z = delta*U_thin.dot(M.dot(Vh_thin))  # Zk tilde, right?
            update_direction = Z-update_Z
            alpha_B = scipy.linalg.inv(delta*u.T.dot(scipy.linalg.inv(update_direction).dot(u))-1)
            
        else: #inside
            idx_max_s = np.argmax(D_thin)
            update_Z = delta*np.outer(U_thin[idx_max_s,:],Vh_thin[idx_max_s,:])
            update_direction = Z-update_Z
            #BINARY SEARCH (xd)
            alpha_B = alpha_binary_search(Z, # This one should be Zk, not Zk tilde... have I chose the correct variable? 
                                          D_thin, # This one shoulde be the direction matrix
                                          delta) 

        nuclear_norm = D_thin.sum()
        U = nuclear_norm*D_thin # standardize the simplex
        r = D_thin.shape
        no_obs = idx_rows.shape[0]
        THRES = 0.001
        
        if ((abs(delta - nuclear_norm) < THRES) and r > 1):
            Z_B = Z + alpha_B*update_direction
            diff_vec_B = Z_B[idx_rows, idx_cols] - X_rated
            
            if 1/(objective_function(diff_vec_B)-low_bound) >= (1/(objective-low_bound)+gamma1/(2*L*D**2)):
                # 1. Move to a lower dimensional face
                Z = Z_B
                #SHOULDN'T THIS BE THE SAME AS THE DENOMINATOR IN THE INEQUALITY CHECK? SEE RED CIRCLES IN IMAGE

            else:
                beta = 0.5 # FIND A GOOD VALUE -- a binary search is also suggested by the paper xdd
                Z_A = Z + beta*update_direction
                diff_vec_A = Z_A[idx_rows, idx_cols] - X_rated
            if 1/(objective_function(diff_vec_A)-low_bound) >= (1/(objective-low_bound)+gamma2/(2*L*D**2)):
                # 2. Stay in the current face
                Z = Z_A
                #SHOULDN'T THIS BE THE SAME AS THE DENOMINATOR IN THE INEQUALITY CHECK? SEE RED CIRCLES IN IMAGE

        else:

            # 3. Do a regular FW step and update the lower bound
            #Zk update
            idx_max_s = np.argmax(D_thin)
            update_Z = -delta*np.outer(U_thin[idx_max_s,:],Vh_thin[:,idx_max_s]) # Am i selecting right the vectors??
            alpha_k = 2/(it+2)
            Z = (1-alpha_k)*Z + alpha_k*update_Z

            # Lower bound update
            direction_vec = update_Z.flatten() - Z.flatten()

            grad = grad.toarray() # this method converts the sparse matrix into a numpy array!

            wolfe_gap = grad.T.flatten() * direction_vec #added the flatten otherwise you can't do the operation
            B_w = objective + wolfe_gap.sum()
            #new_low_bound = np.max(low_bound, B_w)   # gave problems during the execution: wanted both numbers as integers??

            ''' TRIED THIS INSTEAD'''

            if low_bound >= B_w:
              new_low_bound = low_bound
            else:
              new_low_bound = B_w
            ''' '''

        # Loss
        diff_vec = Z[idx_rows, idx_cols] - X_rated
        new_objective = objective_function(diff_vec)

        # Improvement at this iteration
        diff_objective = np.abs(objective - new_objective)
        objective = new_objective

        # Gradient
        grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))

        # Thin SVD
        r_grad = sparse.csgraph.structural_rank(grad)   # Compute rank of the gradient sparse matrix to find thin SVD size
        U_thin, D_thin, Vh_thin = sparse.linalg.svds(grad, k = 1, which='LM')   # Compute k = rank singular values # replaced r_grad with 1

        # Count iteration
        it += 1
        
        print('Iteration:', it, 'f(Z_k):', objective, 'f(Z_{k-1}) -f(Z_k):', diff_objective)

    return Z, objective

In [ ]:
pred_ratings, loss = FW_inface(new_data, FW_objective_function, delta = 7000, max_iter=201)

Iteration: 1 f(Z_k): 3292161.1752353515 f(Z_{k-1}) -f(Z_k): 506335.957855992


## Sub-Chapter